In [ ]:
import nlopt
import numpy as np
import kinpy as kp
import os
def myfunc(x, grad):
  if grad.size > 0:
    grad[0] = 0.0
    grad[1] = 0.5 / np.sqrt(x[1])
  return np.sqrt(x[1])
def myconstraint(x, grad, a, b):
  if grad.size > 0:
    grad[0] = 3 * a * (a*x[0] + b)**2
    grad[1] = -1.0
  return (a*x[0] + b)**3 - x[1]
opt = nlopt.opt(nlopt.LD_MMA, 2)
opt.set_lower_bounds([-float('inf'), 0])
opt.set_min_objective(myfunc)
opt.add_inequality_constraint(lambda x,grad: myconstraint(x,grad,2,0), 1e-8)
opt.add_inequality_constraint(lambda x,grad: myconstraint(x,grad,-1,1), 1e-8)
opt.set_xtol_rel(1e-4)
x = opt.optimize([1.234, 5.678])
minf = opt.last_optimum_value()
print("optimum at ", x[0], x[1])
print("minimum value = ", minf)
print("result code = ", opt.last_optimize_result())

In [ ]:
#urdf_path = '~/Downloads/data/hand/aSampleForearmURDF/aSampleForearm.urdf'
urdf_path = '~/Downloads/data/hand/psyonic/ability_hand_right.urdf'
chain = kp.build_chain_from_urdf(open(os.path.expanduser(urdf_path), 'rb').read())
mimic_joints = {
  'index_q2': ('index_q1', 1.05851325, 0.72349796),
  'middle_q2': ('middle_q1', 1.05851325, 0.72349796),
  'ring_q2': ('ring_q1', 1.05851325, 0.72349796),
  'pinky_q2': ('pinky_q1', 1.05851325, 0.72349796),
}
tip_joints = {
  'thumb_anchor_frame': ['thumb_q1', 'thumb_q2'],
  'index_anchor_frame': ['index_q1', 'index_q2'],
  'middle_anchor_frame': ['middle_q1', 'middle_q2'],
  'ring_anchor_frame': ['ring_q1', 'ring_q2'],
  'pinky_anchor_frame': ['pinky_q1', 'pinky_q2']
}
joint_indices = {name: index for index, name in enumerate(chain.get_joint_parameter_names())}
finger_tips = [
  'thumb_anchor_frame',
  'index_anchor_frame',
  'middle_anchor_frame',
  'ring_anchor_frame'
]
finger_chains = [kp.chain.SerialChain(
  chain, tip_frame, 'thumb_base_frame') for tip_frame in finger_tips]

In [ ]:
human_points = np.array([
 [ 3.2725373e-01,  7.7795637e-01,  5.2816981e-01],   
 [-9.6739009e-02,  7.1368307e-01,  2.6825169e-01],  
 [-3.8875103e-01,  5.4602653e-01, -1.4086844e-02],  
 [-6.6549540e-01,  3.7160176e-01, -2.7358043e-01],   
 [-9.3673939e-01,  1.8285528e-01, -3.0206269e-01],   
 [-2.5174081e-01,  5.9735239e-02,  1.1930531e-03],  
 [-4.6450284e-01, -2.7687421e-01, -1.0340554e-01],   
 [-6.8177640e-01, -4.9950594e-01, -2.0901288e-01],    
 [-8.2444388e-01, -7.3041087e-01, -4.0643957e-01],   
 [-2.1448732e-10, -1.1600489e-09,  1.6249215e-09],                             
 [-1.1306240e-01, -4.0202361e-01, -1.3616551e-01],   
 [-1.8879688e-01, -7.0582879e-01, -2.5055227e-01],                            
 [-2.2684556e-01, -1.0986997e+00, -2.3798983e-01],                       
 [ 2.2809474e-01,  2.9315004e-02, -5.5515267e-02],                                        
 [ 2.2521757e-01, -2.7360198e-01, -1.9113588e-01],                     
 [ 2.2941884e-01, -4.8866022e-01, -3.7809783e-01],   
 [ 1.2185465e-01, -9.6826726e-01, -3.6378175e-01],                                        
 [ 4.7356415e-01,  1.3390288e-01, -1.3963695e-01],                       
 [ 5.4217476e-01, -2.1130911e-01, -2.6199883e-01],   
 [ 5.8714467e-01, -3.7332100e-01, -3.9369622e-01],                                        
 [ 5.9575760e-01, -6.8251586e-01, -4.8331016e-01]])

def vector_length(vector):
  return np.linalg.norm(vector)

def normalize(vector):
  return vector / vector_length(vector)

def weight_and_scale(vector, vector_index):
  length_threshold = 0.1
  length = vector_length(vector)
  antipodal_set = set([4, 5, 6])
  primary_set = set([7, 8, 0])
  if length > length_threshold:
    return 1, 1.6 * length
  elif vector_index in antipodal_set:
    return 200, 1e-4
  elif vector_index in primary_set:
    return 400, 3e-2
  else:
    return 1, 1.6 * length

def human_points_to_vectors(points):
  vectors = [
    points[4, :] - points[0, :],
    points[8, :] - points[0, :],
    points[12, :] - points[0, :],
    points[16, :] - points[0, :],
    points[8, :] - points[4, :],
    points[12, :] - points[4, :],
    points[16, :] - points[4, :],
    points[12, :] - points[8, :],
    points[16, :] - points[8, :],
    points[16, :] - points[12, :]
  ]
  return vectors

def robot_config_to_vectors(joint_positions):
  tip_positions = []
  for index, finger_tip_frame in enumerate(finger_tips):
    finger_joints = [joint_positions[joint_indices[joint_name]]
                     for joint_name in tip_joints[finger_tip_frame]]
    tip_positions.append(finger_chains[index].forward_kinematics(finger_joints).pos)

  vectors = [
    tip_positions[0],
    tip_positions[1],
    tip_positions[2],
    tip_positions[3],
    tip_positions[1] - tip_positions[0],
    tip_positions[2] - tip_positions[0],
    tip_positions[3] - tip_positions[0],
    tip_positions[2] - tip_positions[1],
    tip_positions[3] - tip_positions[1],
    tip_positions[3] - tip_positions[2]
  ]
  return vectors

def compute_jacobians(joint_positions):
  jacobians = []
  for index, finger_tip_frame in enumerate(finger_tips):
    finger_joint_indices = [joint_indices[joint_name] for joint_name in tip_joints[finger_tip_frame]]
    finger_joints = joint_positions[finger_joint_indices]  
    jac = np.zeros((6, len(joint_positions)))
    jac[:, finger_joint_indices] = finger_chains[index].jacobian(finger_joints)
    jacobians.append(jac)
  return jacobians

def metric(joint_positions, gradient):
  d_vector_d_tip_positions = np.array([
    [1, 0, 0, 0],
    [0, 1, 0, 0],
    [0, 0, 1, 0],
    [0, 0, 0, 1],
    [-1, 1, 0, 0],
    [-1, 0, 1, 0],
    [-1, 0, 0, 1],
    [0, -1, 1, 0],
    [0, -1, 0, 1],
    [0, 0, -1, 1]]
  )
  human_vectors = human_points_to_vectors(human_points)
  cost = 0
  num_dof = len(joint_positions)
  vectors = robot_config_to_vectors(joint_positions)
  new_grad = np.zeros((num_dof, ))
  jacobians = compute_jacobians(joint_positions)
  for vector_index, vector in enumerate(vectors):
    weight, scale = weight_and_scale(vector, vector_index)
    diff = vector - scale * normalize(human_vectors[vector_index])
    cost += weight * np.linalg.norm(diff) ** 2
    d_vector_d_joint_position = np.zeros((3, num_dof))
    for joint_index in range(d_vector_d_tip_positions.shape[1]):
      d_vector_d_joint_position += d_vector_d_tip_positions[
        vector_index, joint_index] * jacobians[joint_index][:3, :]
    new_grad += weight * np.dot(diff, d_vector_d_joint_position)
  cost = 0.5 * cost + 2.5e-3 * np.linalg.norm(joint_positions) ** 2
  new_grad += 5e-3 * joint_positions
  gradient[:] = new_grad[:]

  return cost

In [ ]:
jps = np.array([0.1] * 16)
gradient = np.zeros(jps.shape)
cost = metric(jps, gradient)

In [ ]:
opt = nlopt.opt(nlopt.LD_MMA, 1)
opt.set_lower_bounds([-float('inf'), 0])
opt.set_min_objective(myfunc)
opt.add_inequality_constraint(lambda x,grad: myconstraint(x,grad,2,0), 1e-8)
opt.add_inequality_constraint(lambda x,grad: myconstraint(x,grad,-1,1), 1e-8)
opt.set_xtol_rel(1e-4)
x = opt.optimize([1.234, 5.678])
minf = opt.last_optimum_value()

In [ ]:
from matplotlib import pyplot as plt
%matplotlib qt
fig = plt.figure()
ax = fig.add_subplot(projection='3d')
colors = (np.array([list(range(20, -1, -1)), [0] * 21, list(range(0, 21))]).transpose() / 20)#.astype(np.int32)
# ax.scatter(locs[:, 0], locs[:, 1], locs[:, 2], marker=, c=colors, linewidths=10)
for i in range(21):#plot each point + it's index as text above
  ax.scatter(locs[i,0],locs[i,1],locs[i,2],color=colors[i]) 
  ax.text(locs[i,0],locs[i,1],locs[i,2],  '%s' % (str(i)), size=10, zorder=1,color='k') 
ax.set_xlabel('X Label')
ax.set_ylabel('Y Label')
ax.set_zlabel('Z Label')
ax.set_xlim([-1, 1])
ax.set_ylim([-1, 1])
ax.set_zlim([-1, 1])

In [ ]:
print(chain)

In [ ]:
finger_chains[1].forward_kinematics([0.4, 0.3])

In [ ]:
finger_chains[3].jacobian([0.1] * 2)